# Case Study: Black Pavilion with a Rectangular Skyligth

In [1]:
# OS functionalities (to deal with files)
import os

# Enhanced Iteration capabilities (to use cross-products)
import itertools

# Data processing packages
import pandas as pd
import numpy as np

# Global Setup for Processing Data

In this section, we define all the global constants that will be used thourought the script. By defining them up here, we are making this notebook purely parametric and reusable for other projects (as long as the folder structure remains the same).

In [2]:
# Folders Structure
base_folder = os.getcwd()
results_folder = os.path.join(base_folder, "algorithms")
indicators_folder = os.path.join(base_folder, "performance_indicators")
output_folder = "./output/"

# CSV File Configuration
has_header=True
files_sep= ","
file_extension = 'csv'

# Optimization Settings
runs = [1, 2, 3]
nruns = len(runs)
max_evals = 200

## Problem Definition (in the files) 
### Variables
width = 3
length = 4
material = 5
vars_cols = [width, length, material]

### Objectives  
sUDI = 6
cost = 7
objs_cols = [sUDI, cost]

relevant_cols = vars_cols + objs_cols

## Multi-Objective Optimization Algorithms
### Metaheuristics
pop_size = 15
metaheuristics = ["SPEA2", "NSGAII", "SMPSO", "OMOPSO"]

### Model Based (or metamodel)
metamodels_base = ['GPR', 'RF'] 
metamodels_strategies = ["SPEA2", 'NSGAII', "Random"]
metamodels_algorithms = [f"{b}_{s}" for (b, s) in itertools.product(metamodels_base, metamodels_strategies)]

all_algorithms = metaheuristics + metamodels_algorithms
n_algorithms = len(all_algorithms)

### Filenames with the results 
filenames = [f"{a}_results_0{r}.{file_extension}" for (r, a) in itertools.product(runs, all_algorithms)]
filenames

['SPEA2_results_01.csv',
 'NSGAII_results_01.csv',
 'SMPSO_results_01.csv',
 'OMOPSO_results_01.csv',
 'GPR_SPEA2_results_01.csv',
 'GPR_NSGAII_results_01.csv',
 'GPR_Random_results_01.csv',
 'RF_SPEA2_results_01.csv',
 'RF_NSGAII_results_01.csv',
 'RF_Random_results_01.csv',
 'SPEA2_results_02.csv',
 'NSGAII_results_02.csv',
 'SMPSO_results_02.csv',
 'OMOPSO_results_02.csv',
 'GPR_SPEA2_results_02.csv',
 'GPR_NSGAII_results_02.csv',
 'GPR_Random_results_02.csv',
 'RF_SPEA2_results_02.csv',
 'RF_NSGAII_results_02.csv',
 'RF_Random_results_02.csv',
 'SPEA2_results_03.csv',
 'NSGAII_results_03.csv',
 'SMPSO_results_03.csv',
 'OMOPSO_results_03.csv',
 'GPR_SPEA2_results_03.csv',
 'GPR_NSGAII_results_03.csv',
 'GPR_Random_results_03.csv',
 'RF_SPEA2_results_03.csv',
 'RF_NSGAII_results_03.csv',
 'RF_Random_results_03.csv']

In [3]:
# Sanity check: verify the file names are what we expected
filenames[::3]

['SPEA2_results_01.csv',
 'OMOPSO_results_01.csv',
 'GPR_Random_results_01.csv',
 'RF_Random_results_01.csv',
 'SMPSO_results_02.csv',
 'GPR_NSGAII_results_02.csv',
 'RF_NSGAII_results_02.csv',
 'NSGAII_results_03.csv',
 'GPR_SPEA2_results_03.csv',
 'RF_SPEA2_results_03.csv']

## Input/Output (IO) Methods 

In this subsection we create the methods that will be responsible for loading the data from the files. To manipulate the data, we will use pandas.DataFrame data structure. This can be easily manipulated and different statistics can be computed on top of these abstractions. 

In [4]:
def load_results(filenames, base_folder=results_folder, 
                 has_header=True, keep_header=False, 
                 sep=files_sep, usecols=relevant_cols,  
                 max_lines=max_evals):
    """Loads the data from the specified `base_folder` using the `filenames`.
    It assumes the filenames
    """
    read_args = { 
        "header": 'infer' if keep_header else None, 
        "sep": sep,
        "usecols": usecols if usecols else None,
        "skiprows": 1 if has_header and not keep_header else 0,
    }
    filepaths = [os.path.join(base_folder, f) for f in filenames]        
    if max_lines:
        return [pd.read_csv(f, **read_args)[0:max_lines] for f in filepaths]
    else:
        return [pd.read_csv(f, **read_args) for f in filepaths]

In [5]:
# Always confirm whether the results are according to what you expected
examples = load_results(filenames, max_lines=max_evals)
examples[0].head()

,3,4,5,6,7
0,6.0,99.0,0.0,-66.0,10811.4
1,2.0,105.0,1.0,-47.0,9834.5
2,5.0,30.0,1.0,-53.0,6275.0
3,16.0,109.0,1.0,-40.0,14898.9
4,24.0,31.0,2.0,-56.0,10166.4


In [6]:
list(filter(lambda x: x > max_evals, [len(e) for e in examples]))

[]

In [7]:
def get_run_indices(dfs, run, n_algorithms=n_algorithms):
    """Returns the dataframes that correspond to the specified `run`. 
    This assumes that the dataframes are read per run, that means that 
    if we run two algorithms (SMPSO and SPEA2) for 3 runs each, this 
    method assumes that it was read in the following order:
    
    > SMPSO_run1
    > SPEA2_run1
    > SMPSO_run2
    > SPEA2_run2
    > SMPSO_run3
    > SPEA2_run3
    
    In that case, this invocation `get_run_indices(dfs, 1, n_algorithms=2)`
    will return: 
        dfs[0:2] 
    """
    run -= 1
    return dfs[run*n_algorithms:(run+1)*n_algorithms]

## Pareto Dominance Methods
This section contains methods related to the Pareto optimality (or [Pareto Efficiency](https://en.wikipedia.org/wiki/Pareto_efficiency)).

In [8]:
# **IMPORTANT NOTE**: This function assumes that your problem is a minimization problem for every objective dimension.
def weakly_dominates(v0, v1):
    """Computes whether v0 dominates v1, i.e., whether at least one objective
    is better (in this case, smaller) than some other)
    """
    return np.all(v0 <= v1) and np.any(v0 < v1)
    
# Sanity Check (:
print("(Expected) True   (Obtained)", weakly_dominates(np.array([1, 1]), np.array([2, 2])))
print("(Expected) True   (Obtained)", weakly_dominates(np.array([2, 1]), np.array([2, 2])))
print("(Expected) False  (Obtained)", weakly_dominates(np.array([2, 2]), np.array([1, 1]))) 
print("(Expected) True   (Obtained)", weakly_dominates(np.array([1, 2]), np.array([2, 2]))) 
print("(Expected) False  (Obtained)", weakly_dominates(np.array([1, 3]), np.array([3, 1])))
print("(Expected) False  (Obtained)", weakly_dominates(np.array([3, 1]), np.array([1, 3]))) 
print("(Expected) False  (Obtained)", weakly_dominates(np.array([1, 1]), np.array([1, 1])))  

(Expected) True   (Obtained) True
(Expected) True   (Obtained) True
(Expected) False  (Obtained) False
(Expected) True   (Obtained) True
(Expected) False  (Obtained) False
(Expected) False  (Obtained) False
(Expected) False  (Obtained) False


In [9]:
def get_non_dominated(V, dominance=weakly_dominates):
    """Computes the optimal and non-optimal solutions. 
    Optimal solutions are called non-dominated and non-optimal 
    solutions are called denominated.
    """
    nsols, nobjs = V.shape
    
    dominated = np.zeros((nsols, 1))
    dominated_by = np.zeros((nsols, 1))
    for i in range(nsols):
        for j in range(nsols):
            if i != j:
                if dominance(V[j], V[i]):
                    dominated[i] = 1
                    dominated_by[i] = j 
                    break
                    
    return dominated, dominated_by

In [10]:
def add_isdominated_cols(d, cols=objs_cols):
    """Adds to the provided dataframe columns for Pareto optimal solution."""
    df_copy = d.copy()
    A = np.array(df_copy[cols])
    B, C = get_non_dominated(A)
    df_copy["isDominated"] = pd.DataFrame(B, columns=["isDominated"])
    df_copy["dominatedBy"] = pd.DataFrame(C, columns=["dominatedBy"])
    print(df_copy["isDominated"].value_counts())
    return df_copy

In [11]:
def get_combined_PF(dfs, drop_cols=relevant_cols, objs_cols=objs_cols):
    """Computes the combined Pareto front based on a set of input dataframes"""
    all_data = pd.concat(dfs)
    if drop_cols:
        all_data = all_data.drop_duplicates(drop_cols)
    all_data = all_data.reset_index()
    return add_isdominated_cols(all_data, cols=objs_cols)

## General Methods
This section contains general purpose methods that can be used in your scripts.

In [12]:
def get_symmetric(df, cols):
    """Computes the symmetric value of the provided `cols` and returns a 
    copy of the original dataframe where the values of the specified `cols`
    are symmetric."""
    cols = cols if isinstance(cols, (list, tuple)) else [cols]
    copy_df = df.copy()
    for col in cols:
        copy_df[col] = df[col] * -1
    return copy_df

In [13]:
def unfeasible_ratio_iter(pop_size=pop_size):
    """Computes the unfeasible ratio per `pop_size`."""
    results = []
    for i in range(0, len(df), pop_size):
        results.append(df[df[i:i + pop_size] == 'false']['feasible'].count() / pop_size)
    iteration = pd.Series(data = range(1, int(len(df) / pop_size) + 1))
    ratio = pd.Series(data = results)
    ratio_iter = {'Iteration': iteration, 'Ratio Unfeasible': ratio} 
    return pd.DataFrame(ratio_iter)

In [14]:
def get_last_value_iter(df, col_name='hypervolume', iter_size=pop_size):
    """Iterates the values in the `df` in slices of `iter_size` and 
    gets the last value.
    """
    return df[iter_size-1:len(df):iter_size][col_name]

In [15]:
def indicators_results(results_runs):
    """Computes the statistics mean and standard deviation for the specified runs. 
    
    It assumes the `results_run` are organized as follows: 
        results_runs = [indicators_run1, indicators_run2, indicators_run3], 
    where: 
        indicators_run{i} = [indicators_alg_1_run_{i}, ..., indicators_alg_m_run_{i}]
    """
    average_res = [] 
    std_res = [] 
    
    for algorithm_i in range(n_algorithms):
        # Get the same algorithm from all runs (assuming they were collected in the same way)
        algorithm_results = [r[algorithm_i] for r in results_runs] 
        average_res += [pd.concat(algorithm_results, axis=1).mean(axis=1)]
        std_res += [pd.concat(algorithm_results, axis=1).std(axis=1)]
        
    return average_res, std_res

## Data Visualization

In this section, we visualize the data with visual means. 

In [16]:
# Visualization Framework
import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from plotly import tools

try:
    import plotly.plotly as py
    plotly.tools.set_credentials_file(username='username', api_key='api_key')
except: 
    import chart_studio
    import chart_studio.plotly as py
    chart_studio.tools.set_credentials_file(username='username', api_key='api_key')
    
# Print plotly's version
plotly.__version__

'5.1.0'

Let us create the functions that create the graphs

In [17]:
def get_colors(n, colorscale="viridis"): 
    colors = plt.get_cmap(colorscale).colors
    colors_idx = np.linspace(0, len(colors)-1, n, dtype=int)
    
    colors = [colors[idx] for idx in colors_idx]
    colors_str = [f"rgb({r}, {g}, {b})" for (r, g, b) in colors]
    return colors_str

In [18]:
def scatter(data, x=None, y=None, names=all_algorithms, 
            colorscale="viridis", colors=None, 
            mode="markers", marker_size=5.5, ln_width=1.5, 
            layout=None):
    
    def get_x_y(d): 
        if isinstance(d, pd.Series):
            return np.array(d.index) + 1, d.values
        elif isinstance(d, pd.DataFrame):
            return d[x], d[y]
        else: 
            return np.arange(len(d)), d      
        
    # ----------------------------------------------
    # Normalize input data
    # ----------------------------------------------
    data = data if isinstance(data, (list, tuple)) else [data]
    colors = get_colors(len(data), colorscale) if colorscale else colors
    
    # ----------------------------------------------
    # Determine the data types of provided inputs
    # ----------------------------------------------
    x_data = []
    y_data = []
    for d in data:
        xx, yy = get_x_y(d)
        x_data += [xx]
        y_data += [yy]

    
    traces = []
    for (i, (x,y)) in enumerate(zip(x_data, y_data)):
        traces += [
            go.Scatter(
                x = x,
                y = y,
                mode = mode,
                name = names[i],
                marker = dict(
                    # Markers size
                    size = marker_size,
                    color=colors[i],
                ),
                line=dict(
                    width=ln_width,
                    color=colors[i]
            )
            )]
    
    kwargs = {} if not layout else {"layout": layout}
    fig = go.Figure(data=traces, **kwargs)
    return py.iplot(fig, filename='simple_scatter')

### Default Layout

In [19]:
# Default layout for the pareto fronts graphs
layout = go.Layout(
    template="plotly_white",
    autosize=True,
    legend=dict(
        orientation='h'
    ),
    # Define axis
    xaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=False,
        showline=True,
        ticks='',
        showticklabels=True,
        tickformat='.'
    ),
    yaxis=dict(
        autorange=True,        
        showgrid=True,
        zeroline=False,
        showline=True,
        ticks='',
        showticklabels=True,
        tickformat='.'
    )
)

### Pareto Front

This section contains different functions that explore dataframes having information about the non-dominated and the dominated solutions.

In [20]:
def create_pf(pf, name, x, y, nd_color='rgb(0,0,255)', ln_width=1.5, marker_size=5.5, d_color=None): 
    traces = []
    
    # Create the non dominated trace (in a different color, as specified in *nd_color*)
    x_pf = pf[pf['isDominated'] == 0][x] 
    y_pf = pf[pf['isDominated'] == 0][y]
    x_pf, y_pf = zip(*sorted(zip(x_pf, y_pf)))

    traces += [
        go.Scatter(
            x = x_pf,
            y = y_pf,
            mode = 'lines+markers',
            name = name + " NonDominated",
            opacity=1,

            # Layout do marker
            marker=dict(
                color=nd_color,
                size=marker_size
            ),
            line=dict(
                color=nd_color,
                width=ln_width
            )
        )]
    
    if d_color:
        x_npf = pf[pf['isDominated'] == 1][x]
        y_npf = pf[pf['isDominated'] == 1][y]
        
        # Create the dominated trace (in a different color, as specified in *d_color*)
        traces +=[
            go.Scatter(
                x = x_npf,
                y = y_npf,
                mode = 'markers',
                name = name + " Dominated",
                opacity=0.5,

                # Layout do Marker
                marker=dict(
                    #color = d_color,
                    color = nd_color,
                    size=marker_size*0.5,
                )
            )]

    return traces

In [21]:
def get_traces(pfs, x, y, draw_dominated=True,
               names=all_algorithms, colorscale='viridis', colors=None,
               tpf=None, tpf_name="Combined_PF", tpf_color='rgb(0,0,0)', 
               layout=layout):
    
    pfs = pfs if isinstance(pfs, (list, tuple)) else [pfs]
    names = names if isinstance(names, (list, tuple)) else [names]
    n_pfs = len(pfs)
    colors = get_colors(n_pfs, colorscale) if colorscale else colors
    
    traces = []
    
    if tpf is not None:
        traces += create_pf(pf=tpf, name=tpf_name, x=x, y=y, ln_width=4, marker_size=10, nd_color=tpf_color)
    
    for (i, pf) in enumerate(pfs):
        d_color = colors[i] if draw_dominated else None
        traces += create_pf(pf=pf, name=names[i], x=x, y=y, nd_color=colors[i], d_color=d_color)

    fig = go.Figure(data=traces, layout=layout)
    return traces

In [22]:
def create_pfs(pfs, x, y, draw_dominated=True,
               names=all_algorithms, colorscale='viridis', colors=None,
               tpf=None, tpf_name="Combined_PF", tpf_color='rgb(0,0,0)', 
               layout=layout):
    
    traces = get_traces(pfs, x, y, draw_dominated, names, colorscale, colors, tpf, tpf_name, tpf_color, layout)
    
    fig = go.Figure(data=traces, layout=layout)
    return py.iplot(fig, filename='algorithms_pfs_per_run')

---

# Analysis

## Pareto Front Layouts

In [23]:
# Layout for the pareto fronts graphs of the Black Pavilion
layout_BP = go.Layout(
#     template="plotly_white",
    template="ggplot2",
    autosize=False,
    # Define plot size
    width=900, 
    height=540,
    # Legend Position
    legend=dict(
        orientation='h',
        x=-0.01,
        y=-0.2
    ),
    # Define axis
    xaxis=dict(
        title="Cost [€]",
        range=[3500, 11150],
        showgrid=True,
        zeroline=False,
        showline=True,
        ticks='',
        showticklabels=True,
        tickformat='.'
    ),
    yaxis=dict(
        title="sUDI [%]",
        range=[40, 90],    
        showgrid=True,
        zeroline=False,
        showline=True,
        ticks='',
        showticklabels=True,
        tickformat='.'
    )
)

In [24]:
# Layout for the pareto fronts graphs of the Black Pavilion - JUST FOR THE GPR_RANDOM ALGORITHM
layout_Random = go.Layout(
#     template="plotly_white",
    template="ggplot2",
    autosize=False,
    # Define plot size
    width=900, 
    height=540,
    # Legend Position
    legend=dict(
        orientation='h',
        x=0,
        y=-0.2
    ),
    # Define axis
    xaxis=dict(
        title="Cost [€]",
        range=[3500, 18000],
        showgrid=True,
        zeroline=False,
        showline=True,
        ticks='',
        showticklabels=True,
        tickformat='.'
    ),
    yaxis=dict(
        title="sUDI [%]",
        range=[0, 90],    
        showgrid=True,
        zeroline=False,
        showline=True,
        ticks='',
        showticklabels=True,
        tickformat='.'
    )
)

## Read Algorithms

In [25]:
# Read algorithms  
dfs = load_results(filenames)

In [26]:
# Compute non_dominated_solutions (per run)
pfs = [add_isdominated_cols(df) for df in dfs]

1.0    162
0.0     38
Name: isDominated, dtype: int64
1.0    178
0.0     22
Name: isDominated, dtype: int64
1.0    177
0.0     23
Name: isDominated, dtype: int64
1.0    173
0.0     27
Name: isDominated, dtype: int64
1.0    177
0.0     23
Name: isDominated, dtype: int64
1.0    131
0.0     15
Name: isDominated, dtype: int64
1.0    186
0.0     14
Name: isDominated, dtype: int64
1.0    165
0.0     35
Name: isDominated, dtype: int64
1.0    177
0.0     23
Name: isDominated, dtype: int64
1.0    180
0.0     20
Name: isDominated, dtype: int64
1.0    177
0.0     23
Name: isDominated, dtype: int64
1.0    174
0.0     26
Name: isDominated, dtype: int64
1.0    172
0.0     28
Name: isDominated, dtype: int64
1.0    162
0.0     38
Name: isDominated, dtype: int64
1.0    183
0.0     17
Name: isDominated, dtype: int64
1.0    170
0.0     30
Name: isDominated, dtype: int64
1.0    174
0.0     26
Name: isDominated, dtype: int64
1.0    169
0.0     31
Name: isDominated, dtype: int64
1.0    168
0.0     32
Name: 

In [27]:
# Computes combined Pareto Front (optimal solutions found from all the algorithms, all the runs)
combined_pf = get_combined_PF(dfs, drop_cols=relevant_cols)

1.0    3007
0.0      22
Name: isDominated, dtype: int64


In [28]:
# Since sUDI is actually a maximization, let us use the symmetric operation 
pfs = [get_symmetric(pf, sUDI) for pf in pfs]
combined_pf = get_symmetric(combined_pf, sUDI)

## Plot all obtained solution (1 single plot for all algorithms, all runs) 

In [29]:
create_pfs(pfs, x=cost, y=sUDI, tpf=combined_pf, names=filenames, colorscale='plasma', draw_dominated=False, layout=layout_BP)

In [30]:
create_pfs(pfs, x=cost, y=sUDI, tpf=combined_pf, names=filenames, draw_dominated=True, layout=layout_BP)

## Plot algorithms per run (3 plot one for each run)

In [31]:
# Get Run 1 
dfs_run_1 = get_run_indices(dfs, 1)
pfs_run_1 = [add_isdominated_cols(df) for df in dfs_run_1]
combined_PF_run_1 = get_combined_PF(pfs_run_1)

1.0    162
0.0     38
Name: isDominated, dtype: int64
1.0    178
0.0     22
Name: isDominated, dtype: int64
1.0    177
0.0     23
Name: isDominated, dtype: int64
1.0    173
0.0     27
Name: isDominated, dtype: int64
1.0    177
0.0     23
Name: isDominated, dtype: int64
1.0    131
0.0     15
Name: isDominated, dtype: int64
1.0    186
0.0     14
Name: isDominated, dtype: int64
1.0    165
0.0     35
Name: isDominated, dtype: int64
1.0    177
0.0     23
Name: isDominated, dtype: int64
1.0    180
0.0     20
Name: isDominated, dtype: int64
1.0    1277
0.0      22
Name: isDominated, dtype: int64


In [32]:
# Get Run 2 
dfs_run_2 = get_run_indices(dfs, 2)
pfs_run_2 = [add_isdominated_cols(df) for df in dfs_run_2]
combined_PF_run_2 = get_combined_PF(pfs_run_2)

1.0    177
0.0     23
Name: isDominated, dtype: int64
1.0    174
0.0     26
Name: isDominated, dtype: int64
1.0    172
0.0     28
Name: isDominated, dtype: int64
1.0    162
0.0     38
Name: isDominated, dtype: int64
1.0    183
0.0     17
Name: isDominated, dtype: int64
1.0    170
0.0     30
Name: isDominated, dtype: int64
1.0    174
0.0     26
Name: isDominated, dtype: int64
1.0    169
0.0     31
Name: isDominated, dtype: int64
1.0    168
0.0     32
Name: isDominated, dtype: int64
1.0    187
0.0     13
Name: isDominated, dtype: int64
1.0    1359
0.0      22
Name: isDominated, dtype: int64


In [33]:
# Get Run 3 
dfs_run_3 = get_run_indices(dfs, 3)
pfs_run_3 = [add_isdominated_cols(df) for df in dfs_run_3]
combined_PF_run_3 = get_combined_PF(pfs_run_3)

1.0    180
0.0     20
Name: isDominated, dtype: int64
1.0    182
0.0     18
Name: isDominated, dtype: int64
1.0    183
0.0     17
Name: isDominated, dtype: int64
1.0    162
0.0     38
Name: isDominated, dtype: int64
1.0    179
0.0     21
Name: isDominated, dtype: int64
1.0    175
0.0     25
Name: isDominated, dtype: int64
1.0    184
0.0     16
Name: isDominated, dtype: int64
1.0    132
0.0     18
Name: isDominated, dtype: int64
1.0    168
0.0     32
Name: isDominated, dtype: int64
1.0    183
0.0     17
Name: isDominated, dtype: int64
1.0    1342
0.0      21
Name: isDominated, dtype: int64


In [34]:
# Compute combined Pareto Front for run 1
pfs_run_1 = [get_symmetric(df, sUDI) for df in pfs_run_1]
combined_PF_run_1 = get_symmetric(combined_PF_run_1, sUDI)

In [35]:
# Compute combined Pareto Front for run 2
pfs_run_2 = [get_symmetric(df, sUDI) for df in pfs_run_2]
combined_PF_run_2 = get_symmetric(combined_PF_run_2, sUDI)

In [36]:
# Compute combined Pareto Front for run 3
pfs_run_3 = [get_symmetric(df, sUDI) for df in pfs_run_3]
combined_PF_run_3 = get_symmetric(combined_PF_run_3, sUDI)

In [37]:
# Get All Runs
# Before running this run the previous cells (the ones that define dfs_run_{i})
pfs_all_runs = []
for i in range(n_algorithms):
    pf=get_combined_PF([dfs_run_1[i], dfs_run_2[i], dfs_run_3[i]])
    pf=get_symmetric(pf, sUDI)
    pfs_all_runs.append(pf)

1.0    427
0.0     21
Name: isDominated, dtype: int64
1.0    445
0.0     20
Name: isDominated, dtype: int64
1.0    412
0.0     18
Name: isDominated, dtype: int64
1.0    294
0.0     21
Name: isDominated, dtype: int64
1.0    449
0.0     18
Name: isDominated, dtype: int64
1.0    425
0.0     17
Name: isDominated, dtype: int64
1.0    558
0.0     12
Name: isDominated, dtype: int64
1.0    437
0.0     22
Name: isDominated, dtype: int64
1.0    442
0.0     22
Name: isDominated, dtype: int64
1.0    484
0.0     19
Name: isDominated, dtype: int64


### Run 1

In [38]:
# Create plot with combined pareto front retrieved only from all algorithms of run 1
create_pfs(pfs_run_1, x=cost, y=sUDI, tpf=combined_PF_run_1, names=all_algorithms, draw_dominated=True)

In [39]:
# Create plot with combined pareto front retrieved from all runs
create_pfs(pfs_run_1, x=cost, y=sUDI, tpf=combined_pf, names=all_algorithms, draw_dominated=True)

### Run 2

In [40]:
# Create plot with combined pareto front retrieved only from all algorithms of run 2
create_pfs(pfs_run_2, x=cost, y=sUDI, tpf=combined_PF_run_2, names=all_algorithms, draw_dominated=True, layout=layout_BP)

In [41]:
# Create plot with combined pareto front retrieved from all runs
create_pfs(pfs_run_2, x=cost, y=sUDI, tpf=combined_pf, names=all_algorithms, draw_dominated=True, layout=layout_BP)

### Run 3

In [42]:
# Create plot with combined pareto front retrieved only from all algorithms of run 3
create_pfs(pfs_run_3, x=cost, y=sUDI, tpf=combined_PF_run_3, names=all_algorithms, draw_dominated=True, layout=layout_BP)

In [43]:
# Create plot with combined pareto front retrieved from all runs
create_pfs(pfs_run_3, x=cost, y=sUDI, tpf=combined_pf, names=all_algorithms, draw_dominated=True, layout=layout_BP)

### All Runs

In [44]:
# Create plot with one PF per algorthim, using the information of the 3 runs
create_pfs(pfs_all_runs, x=cost, y=sUDI, tpf=None, names=all_algorithms, 
           colorscale=None,
           colors=['#93e0ed', '#307382', '#2fcce0', '#042780', 
                   '#fcb447', '#ff8000', '#db3f30', 
                   '#de7ec9', '#8a39db', '#db30b9'],
           draw_dominated=False, layout=layout_BP)

In [45]:
# Create plot with one PF per algorthim, using the information of the 3 runs and with the combined PF
create_pfs(pfs_all_runs, x=cost, y=sUDI, tpf=combined_pf, names=all_algorithms, 
           colorscale=None,
           colors=[
                '#2fcce0', '#307382',
                '#3FC283', '#046645',
                '#ffc800', '#ff8000', '#db3f30', 
                '#de7ec9', '#db30b9', '#8a39db'],
           draw_dominated=True, layout=layout_BP)

### The 3 Runs of GPR Random
The GPR Random algorithm was the one that showed the worse results for this particular case study, in order to analyse this algorithm in more detail and to directly compare all the runs, the next plots shows the combined results of the three runs for this algorithm. This way it is easier to understand the bad performance for the GPR Random algorithm showed in the plot with the hypervolume average per iteration.

#### 3 plots, 1 for each run

In [46]:
# Create plot with combined pareto front and the Pareto front of the GPR_Random run 1
create_pfs(pfs_run_1[6], x=cost, y=sUDI, tpf=combined_pf, names=all_algorithms[6],
           colorscale=None, colors=['#db3f30'],
           draw_dominated=False, layout=layout_Random)

In [47]:
# Create plot with combined pareto front and the Pareto front of the GPR_Random run 2
create_pfs(pfs_run_2[6], x=cost, y=sUDI, tpf=combined_pf, names=all_algorithms[6],
           colorscale=None,
           colors=['#db3f30'],
           draw_dominated=False, layout=layout_Random)

In [48]:
# Create plot with combined pareto front and the Pareto front of the GPR_Random run 3
create_pfs(pfs_run_3[6], x=cost, y=sUDI, tpf=combined_pf, names=all_algorithms[6], 
           colorscale=None,
           colors=['#db3f30'],
           draw_dominated=False, layout=layout_Random)

#### Plot with all the 3 runs

In [50]:
traces_run1 = lambda color: get_traces(pfs_run_1[6], x=cost, y=sUDI, tpf=combined_pf, names=all_algorithms[6], 
                                       colorscale=None,
                                       colors=[color],
                                       draw_dominated=False)

traces_run2 = lambda color: get_traces(pfs_run_2[6], x=cost, y=sUDI, tpf=combined_pf, names=all_algorithms[6], 
                                       colorscale=None,
                                       colors=[color],
                                       draw_dominated=False)

traces_run3 = lambda color: get_traces(pfs_run_3[6], x=cost, y=sUDI, tpf=combined_pf, names=all_algorithms[6], 
                                       colorscale=None,
                                       colors=[color],
                                       draw_dominated=False)

In [51]:
# Creates Combined PF and the PFs for each run of the GPR Random Algorithm in three different subplots
fig=plotly.subplots.make_subplots(rows=1, cols=3, subplot_titles=('Run 1', 'Run 2', 'Run 3'))

color='#db3f30'

fig.append_trace(traces_run1(color)[0], 1, 1)
fig.append_trace(traces_run1(color)[1], 1, 1)
fig.append_trace(traces_run2(color)[0], 1, 2)
fig.append_trace(traces_run2(color)[1], 1, 2)
fig.append_trace(traces_run3(color)[0], 1, 3)
fig.append_trace(traces_run3(color)[1], 1, 3)

axis_x = xaxis=dict(title="Cost [€]",
                    range=[3500, 18000],
                    showgrid=True,
                    zeroline=False,
                    showline=True,
                    ticks='',
                    showticklabels=True,
                    tickformat='.')
axis_y = yaxis2=dict(title="sUDI [%]",
                     range=[0, 90],
                     showgrid=True,
                     zeroline=False,
                     showline=True,
                     ticks='',
                     showticklabels=True,
                     tickformat='.')

fig['layout'].update(height=500, width=1200,
                     template="plotly_white",
                     legend=dict(
                         orientation='h',
                         y=-0.2
                     ),
                     xaxis1=axis_x,
                     yaxis1=axis_y,
                     xaxis2=axis_x,
                     yaxis2=axis_y,
                     xaxis3=axis_x,
                     yaxis3=axis_y)

py.iplot(fig)

## Plot the Performance Indicators per iteration
After running the optimization processes for the 10 algorithms for 3 runs, we computed their __Hypervolume (HV)__ \[1\], as well as other indicators, such as the __Overall Non-dominated Vector Generated (ONVG)__ \[2\] and the __Set Spacing (SS)__ \[3\].

In [52]:
idfs = load_results(filenames, base_folder=indicators_folder, has_header=True, keep_header=True, usecols=None)

In [53]:
idfs[0].head() # Sanity Check

,hypervolume,spacing,onvg
0,0.383592,-1.000000,1.0
1,0.419201,0.000000,2.0
2,0.572020,0.000000,2.0
3,0.572020,0.000000,2.0
4,0.573661,0.080999,3.0


In [54]:
idfs_run_1 = get_run_indices(idfs, 1)
idfs_run_2 = get_run_indices(idfs, 2)
idfs_run_3 = get_run_indices(idfs, 3)

In [55]:
hypervolume_run_1 = [get_last_value_iter(df, col_name='hypervolume') for df in idfs_run_1]
hypervolume_run_2 = [get_last_value_iter(df, col_name='hypervolume') for df in idfs_run_2]
hypervolume_run_3 = [get_last_value_iter(df, col_name='hypervolume') for df in idfs_run_3]
hypervolume_per_run = [hypervolume_run_1, hypervolume_run_2, hypervolume_run_3]

In [56]:
onvg_run_1 = [get_last_value_iter(df, col_name='onvg') for df in idfs_run_1]
onvg_run_2 = [get_last_value_iter(df, col_name='onvg') for df in idfs_run_2]
onvg_run_3 = [get_last_value_iter(df, col_name='onvg') for df in idfs_run_3]
onvg_per_run = [onvg_run_1, onvg_run_2, onvg_run_3]

In [57]:
spacing_run_1 = [get_last_value_iter(df, col_name='spacing') for df in idfs_run_1]
spacing_run_2 = [get_last_value_iter(df, col_name='spacing') for df in idfs_run_2]
spacing_run_3 = [get_last_value_iter(df, col_name='spacing') for df in idfs_run_3]
spacing_per_run = [spacing_run_1, spacing_run_2, spacing_run_3]

### Run 1

In [58]:
scatter(hypervolume_run_1, mode='lines+markers')

In [59]:
scatter(onvg_run_1, mode='lines+markers')

In [60]:
scatter(spacing_run_1, mode='lines+markers')

### Run 2

In [61]:
scatter(hypervolume_run_2, mode='lines+markers')

In [62]:
scatter(onvg_run_2, mode='lines+markers')

In [63]:
scatter(spacing_run_2, mode='lines+markers')

### Run 3

In [64]:
scatter(hypervolume_run_3, mode='lines+markers')

In [65]:
scatter(onvg_run_3, mode='lines+markers')

In [66]:
scatter(spacing_run_3, mode='lines+markers')

## Average and Standard Deviation across the 3 runs

In [67]:
# Default layout for the pareto fronts graphs
layout_indicators = lambda x: go.Layout(
        template="plotly_white",
        autosize=False,
        # Define plot size
        width=850,
        height=500,
        # Legend Position
        legend=dict(
            orientation='h',
            x=0.02,
            y=-0.25
        ),
        # Define axis
        xaxis=dict(
            title="Number of Evaluations",
            range=[1, 230],
            # autorange=True,
            showgrid=True,
            zeroline=True,
            showline=True,
            ticks='',
            showticklabels=True,
            tickvals = np.array(range(pop_size, 225, pop_size)),
        ),
        yaxis=dict(
            title=x,
            showgrid=True,
            zeroline=False,
            showline=True,
            ticks='',
            showticklabels=True
        )
)

### Hypervolume Indicator

In [68]:
layout_hv = go.Layout(
    template="ggplot2",
    
    # Define plot size
    autosize=False,
#     width=850, 
#     height=600,
    
    width=600, 
    height=500,

    legend=dict(
            orientation='h',
            x=0.02,
            y=-0.25
        ),

    # Define axis
    xaxis=dict(
        title="Number of Evaluations",
        range=[10, 200],
        showgrid=True,
        zeroline=True,
        showline=True,
        ticks='',
        showticklabels=True,
        tickvals = np.array(range(pop_size, 225, pop_size)),
    ),
    yaxis=dict(
        title="Hypervolume",
        range=[0.35, 0.75],    
        showgrid=True,
        zeroline=False,
        showline=True,
        ticks='',
        showticklabels=True,
#         tickformat='.',
        tick0 = 0.6,
        dtick = 0.1
    )
)

In [69]:
hypervolume_avg, hypervolume_std = indicators_results(hypervolume_per_run)

In [70]:
scatter(hypervolume_avg, mode='lines+markers',
#         layout=layout_indicators('Hypervolume'),
        layout=layout_hv,
        colorscale=None,
        colors=[# '#93e0ed', '#307382', '#2fcce0', '#042780',
                '#2fcce0', '#307382',
                '#3FC283', '#046645',
                '#ffc800', '#ff8000', '#db3f30', 
                '#de7ec9', '#db30b9', '#8a39db']
       )

In [71]:
scatter(hypervolume_std, mode='lines+markers', layout=layout_indicators('Hypervolume'))

### ONVG Indicator

In [72]:
onvg_avg, onvg_std = indicators_results(onvg_per_run)

In [73]:
scatter(onvg_avg, mode='lines+markers', layout=layout_indicators('ONVG'))

In [74]:
scatter(onvg_std, mode='lines+markers', layout=layout_indicators('ONVG'))

### Spacing Indicator

In [75]:
spacing_avg, spacing_std = indicators_results(spacing_per_run)

In [76]:
scatter(spacing_avg, mode='lines+markers', layout=layout_indicators('Spacing'))

In [77]:
scatter(spacing_std, mode='lines+markers', layout=layout_indicators('Spacing'))

__References__

\[1\] Russo, L. M. S., & Francisco, A. P. (2014). _Extending quick hypervolume_. Journal of Heuristics, 22(3), 245–271.

\[2\] Veldhuizen, D. V. (1999). _Multi Objective evolutionary algorithms: Classifications, Analysis, New Innovations. Multi Objective evolutionary algorithms_. Air Force Institute of Technology, Wright Patterson, Ohio.

\[3\] Schott, J. R. (1995). _Fault Tolerant Design Using Single and Multicriteria Genetic Algorithm Optimization_. Massachusetts Institute of Technology, Boston, MA.